[![View On GitHub](https://img.shields.io/badge/View_in_Github-grey?logo=github)](https://github.com/sarus-tech/dp-testing/blob/main/examples/showcase.ipynb)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sarus-tech/dp-testing/blob/main/examples/showcase.ipynb)


# A Practical Method for Testing Differential Privacy

At sarus we build privacy safe analytics tools and one of our initiatives is an open source library called [Qrlew](https://qrlew.readthedocs.io/en/latest/). Qrlew is used to turn sql queries into privacy-safe ones under the framework of [Differential Privacy (DP)](https://en.wikipedia.org/wiki/Differential_privacy). As the library was maturing and more features were added we needed to test that the results generated from DP rewritten queries were actually coherent with the theory. Thanks to this approach, we identified unexpected behavior in one of the mechanisms implemented in Qrlew. This discovery strengthens our confidence in the robustness of the privacy guarantees provided by Qrlew, as it highlights the effectiveness of our testing methodology in identifying and addressing potential issues. This motivated us to open source our testing methodology and tools to the developers and researchers community so, let's dive in.

To build trustworthy data analysis systems, it’s important to verify that differential privacy mechanisms truly provide the privacy guarantees they promise. While there have been other attempts to create open-source libraries for differential privacy testing—such as Google’s Differential Privacy [Stochastic Tester](https://github.com/google/differential-privacy/blob/main/cc/testing/README.md), [DP Auditorium](https://github.com/google/differential-privacy/tree/main/python/dp_auditorium), and potentially others, our approach focuses on making it exceptionally easy to create datasets that resemble real-life scenarios and to effortlessly test them with any SQL query. In this ready-to-run Jupyter notebook on Google Colab, we introduce practical methods for testing differential privacy results. Adopting the perspective of an adversary attempting to breach privacy, we demonstrate how hypothesis testing can assess whether a mechanism effectively protects individual entries in a dataset.

Here, we also introduce the dp_tester repository that implements these testing techniques, showcasing how straightforward it is to apply them to your own differential privacy mechanisms. Through hands-on code examples and interactive experiments, you’ll see how easy it is to empirically validate and verify the privacy properties of your implementations using our tools.

This practical approach bridges the gap between theoretical guarantees and real-world applications, empowering developers and researchers to ensure their differential privacy solutions are both robust and effective—all within a convenient and user-friendly Colab environment.

---

## Table of Contents

- [How to test DP results](#how-to-test-dp-results)
- [Experimental settings & Results](#experimental-settings--results)
  1) [Dataset Generation](#dataset-generation)
  2) [Collecting DP Results](#collecting-dp-results)
  3) [Partitioning Results](#partitioning-results)
  4) [Compute Empirical Epsilons](#compute-empirical-epsilons)
- [Conclusions](#conclusions)
- [References](#references)

## How to Test DP Results

Differential Privacy (DP) ensures that the output of a mechanism $ℳ$ does not significantly depend on whether any single individual is included or excluded from the dataset. Formally, a mechanism $ℳ$ is ($\epsilon,\delta$)-differentially private if:

$$\Pr[ℳ(D_0) \in S] \leq e^{\epsilon} \Pr[ℳ(D_1) \in S] + \delta$$
$\forall \; D_0$ and $D_1$, where $|D_0-D_1| \leq 1$ (neighboring datasets differing by at most one individual), $\forall S$. $\epsilon$ is the privacy loss parameter, and $\delta$ is a small failure probability. 

### Adversary’s Perspective

To verify if a mechanism truly satisfies differential privacy, consider an adversary trying to determine if a particular user’s data was included. The adversary faces two situations:
- $D_0$: Dataset with the user included.
- $D_1$: Dataset with the user excluded.

The adversary’s goal is to distinguish which dataset produced the observed sql query result $r \in 𝓡$. We can treat this distinction as a hypothesis testing problem:
- Null Hypothesis ($H_0$): The output $r$ is generated from $ℳ(D_0)$.
- Alternative Hypothesis ($H_1$): The output $r$ is generated from $ℳ(D_1)$.

The [Neyman–Pearson lemma](https://en.wikipedia.org/wiki/Neyman%E2%80%93Pearson_lemma) is a fundamental result in statistical hypothesis testing that provides a method for constructing the most powerful test for distinguishing between two simple hypotheses and has applications across different domains such as in medicine, physics, economy etc. The Neyman–Pearson lemma tells us that the most powerful test (one that best distinguishes $H_1$ from $H_0$) for a given error rate is based on the likelihood ratio:

$$\Lambda(r) = \frac{\rho(r | D_0)}{\rho(r | D_1)}$$

where $\rho(r | D_0)$ and $\rho(r | D_1)$ are the probability densities of $r$ under $H_0$ and $H_1$, respectively. If $\Lambda(r) \geq \tau$ for some threshold $\tau$, the test rejects $H_0$ (i.e., it guesses the dataset is $D_1$).

### Defining False Positives and False Negatives

In our scenario:
- A False Positive ($FP$) occurs if we reject $H_0$ when it is actually true. Under $H_0$, this happens when $\Lambda(r)$ is above the threshold $\tau$:

$$ FP_{\tau} = \Pr_{H_0}[\Lambda(r) \geq \tau] $$
- A False Negative ($FN$) occurs if we fail to reject $H_0$ when $H_1$ is true. Under $H_1$, this means $\Lambda(r)$ is below $\tau$:
 
$$ FN_{\tau} = \Pr_{H_1}[\Lambda(r) < \tau] $$

We can equivalently express these quantities in terms of $S_{\tau}$ defined as $S_{\tau} = \{ r \in S : \Lambda(r) < \tau \}$:

$$ FP_{\tau} = 1 - \Pr_{H_0}[r \in S_{\tau}], \quad FN_{\tau} = \Pr_{H_1}[r \in S_{\tau}] $$

In the graph below there is a schematic representation of our $FP$ and $FN$ definitions:

<div style="text-align: center;">
    <img src="figure_fp-fn.png" width="400" alt="" />
</div>

### From $FP$, $FN$ to an Empirical $\epsilon$

We can rewrite the definition of DP in terms of $FP_{\tau}$ and $FN_{\tau}$ where:
- $\Pr[ℳ(D_0) \in S_{\tau}] = 1 - FP_{\tau}$
- $\Pr[ℳ(D_1) \in S_{\tau}] = FN_{\tau}$

and after re-arranging we have the expression: 

$$\epsilon \geq \ln\left(\frac{1 - \delta - FP_{\tau}}{FN_{\tau}}\right)  \quad \quad \forall \; D_0, D_1, |D_0-D_1| \leq 1, \quad \forall \; S_{\tau}  $$

### Approximations

#### $FP_{\tau}$ and $FN_{\tau}$:
We cannot compute  $FP_{\tau}$  and  $FN_{\tau}$  directly, as they require integrating the unknown continuous probability densities $\rho(r \mid D_0)$  and  $\rho(r \mid D_1)$. However, we can estimate them by approximating these integrals with sums over observed counts, as described below.

We partition the result space $𝓡$ into discrete subsets called buckets. Any result $r$ of an SQL query can be mapped to one and only one bucket $I_j$. Here, we introduce the bucket function $p : 𝓡 \to \mathbb{N}$, which maps each result $r \in 𝓡$ to the integer index $j$ of its corresponding bucket $I_j$. Formally: $p(r) = j \in \mathbb{N}$ if $r \in I_j$. The reciprocal image $p^{-1}(j)$ establishes a bijection between query result sets and the integers they map to: $p^{-1}(j) = \{r \in \mathcal{R} : p(r) = j\}$, ensuring that each integer maps to a unique bucket of results and vice versa.

As an example, queries returning a single numerical value, the buckets $I_j$ can be disjoint intervals but they can also be: $I_1$: [-1 1], $I_2$: [-2 -1] U [1 2], $I_3$: [-3 -2] U [2 3], $I_4$: (-inf -3] U [3 inf) etc. As another more complex example, let's consider the following query:

```SELECT gender, COUNT(*) FROM users GROUP BY gender.```

The result of this query might be seen a mapping $r: G \to \mathbb{N}$, where $G = \{\text{male}, \text{female}\}$ represents the groups. The following is an example of legitimate partition of the results:
- $I_1$: $g \notin \{\text{male}, \text{female}\}$, invalid group,
- $I_2$: $r(g) \notin I_1 \land r(\text{male}) = \emptyset \cup r(\text{female}) = \emptyset$, results are not associated to bucket $I_1$ and one group is not represented,
- $I_3$: $r(g) \notin I_i, \forall i \in \{1, 2\} \land r(\text{male}) > 2*r(\text{female})$, results are not associated to buckets $I_1$ and $I_2$ and there are 2 times more man,
- $I_4$: $r(g) \notin I_i, \forall i \in \{1, 2, 3\} \land r(\text{female}) > 2*r(\text{male})$, results are not associated to buckets $I_1$, $I_2$ and $I_3$ and there are 2 times more female,
- $I_5$: $r(g) \notin I_i, \forall i \in \{1, 2, 3, 4\} \land r(\text{male}) \in [0, 10]$, results are not associated to buckets $I_1$, $I_2$, $I_3$ and $I_4$ and there are between 0 and 10 man,
- $I_6$: $r(g) \notin I_i, \forall i \in \{1, 2, 3, 4, 5\}$, all results that don't satisfy the conditions the other buckets.

We assume that within each bucket, the densities $\rho(r \mid D_0)$ and $\rho(r \mid D_1)$ are constant.
Let  $c_j^{D_0}$  and  $c_j^{D_1}$  be the counts of $C$ experiments over $D_0$ and $D_1$ respectively falling into bucket $I_j$.
The set $S_\tau$, where the likelihood ratio is below the threshold $\tau$, becomes: $S_\tau = \bigcup_{j \in 𝒩\tau} I_j$ where: $𝒩\tau = \left\{ j \in \mathbb{N} : \hat{\Lambda}_j < \tau \right\}.$

The likelihood ratio for bucket $I_j$ is estimated as:

$$\widehat{\Lambda}_j = \frac{c_j^{D_0}}{c_j^{D_1}}$$

We choose $\tau$ to align with the edges of the bucket for convenience in order to identify buckets where the likelihood ratio is below $\tau$.
Under $H_0$, the probability of $r$ falling into the selected buckets is approximated using counts:

$$\widehat{\Pr}_{H_0}[r \in S\tau] = \sum_{j \in 𝒩_\tau} \frac{c_j^{D_0}}{C}$$

Similarly, under $H_1$ we have:
$$\widehat{\Pr}_{H_1}[r \in S\tau] = \sum_{j \in 𝒩_\tau} \frac{c_j^{D_1}}{C}$$

The false positive and false negative rates are then:

$$\widehat{FP}\tau = 1 - \widehat{\Pr}_{H_0}[r \in S_\tau], \quad \widehat{FN}\tau = 1 - \widehat{\Pr}_{H_1}[r \in S_\tau]$$

By the law of large numbers, as the number of experiments increases (typically 10k–50k is sufficient), the estimates $\widehat{FP}_\tau$ and $\widehat{FN}_\tau$ will converge to  the expected values of $FP_\tau$ and $FN_\tau$, respectively, minimizing the estimation error.

#### Neighbor datasets:
We should consider all possible neighboring datasets; however, for practical reasons, we may select only a subset of pairs to test. This means that even if the test passes for the chosen pairs of neighboring datasets, it could still fail for other untested pairs.

Considering the approximations used so far the empirical test 

$$ \epsilon \stackrel{?}{\geq} \hat{\epsilon^{*}} = \max_{\tau, \pi} \left[ ln\left(\frac {1 - \delta - \widehat{FP_{\tau}}} {\widehat{FN_{\tau}}}\right) \right] \quad \quad \forall \; D_0, D_1 \in \{\pi_1, \pi_2, ... \pi_n\}, \pi_i = (D^i_0-D^i_1), |D^i_0-D^i_1| \leq 1, \quad 
\forall \; S_{\tau}  $$

The $?$ indicates that this formula may not hold if the number of experiments is too low.

### Interpreting the Results

- Test Fails ($\hat{\epsilon}^{*} > \epsilon$): The mechanism may leak more information than allowed by $\epsilon$, indicating a potential implementation flaw.
- Test Passes ($\hat{\epsilon}^{*} \leq \epsilon$): The mechanism appears to respect the claimed differential privacy parameters for the tested dataset pairs.

Note that passing the test does not guarantee DP in all cases—only for the tested datasets and queries. However, failing the test is a strong indication that something is wrong and should be investigated further.

## Experimental settings

In this section we go from theory to practice in 4 steps:
1) **Dataset Generation**: we provide a description of the datasets created for the experiments.
2) **Collecting DP Results**: description on how we collected our DP results.
3) **Partitioning Results**: description of the how we partitioned the results into buckets needed for computing $FN$ and $FP$.
4) **Compute Empirical Epsilons**: computing the empirical epsilon and compare it to the actual epsilon to see if our results are coherent with the theory.

### Dataset Generation

In the following cells we create a postgres database and we push our $D_0$ and $D_1$ datasets which differ by the removal of the data relative to one user.

The dataset consists in 2 tables:
- `users`: it has 100 lines each related to a distinct user. The columns are `id: (int) UNIQUE` identifying the user and `income: (float)` is a random number drawn from $𝒩(\mu=40000, \sigma=10000)$
- `transactions`: it has 10000 entries, The columns are `id: (int) UNIQUE` the transaction id, `user_id: (int)` the user who is making the transaction (it is one of the user in the `users` table), `spent: (float)` is a random number drawn from $U(a=5,b=500)$, `store_id: (int)` identifier of the store, there are 200 unique stores, `other_id: (int)` is the identifier of some arbitrary feature of the user, there are 10 possible unique other ids.


In the `transactions` table for each user are recorder up to 500 transactions. User 0 is the one making the most of them (it does exactly 500 transactions) while the remaining transactions are split uniformly among the remaining users. Moreover, the user 0 likes to make transactions in all stores and he likes them differently: the higher the store_id the higher the frequency he visits the store to buy something. In other words, user 0 affects all the stores in different way. The other users don't have any particular preference among the stores so they pick them randomly. Lastly, imagine the `other_id` is associated to a characteristic of the user such as it's actual age for instance, thus to each user is associated only one `other_id`.

$D_0$ is pushed to the default postgres schema while $D_1$ in which all the information about the user 0 has been removed is pushed to the tables in the `D_1` schema. Here we want to know how much it has been spent in each store so we will collect privacy-safe results from this query:

```
"SELECT store_id, SUM(spent) AS spent_per_store FROM transactions GROUP BY store_id"
```

Notice that submitting this to $D_1$ where the user affecting all `store_id`s is removed is the worst scenario.

---

In [ ]:
%%capture
# Create a postgres database
# Inspired by https://colab.research.google.com/github/tensorflow/io/blob/master/docs/tutorials/postgresql.ipynb#scrollTo=YUj0878jPyz7
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql-14
# Start postgresql server
!sudo sed -i "s/port = 5432/port = 5433/g" /etc/postgresql/14/main/postgresql.conf
!sudo service postgresql start
# Set password
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'pyqrlew-db'"
!pip install git+https://github.com/sarus-tech/dp-testing.git

In [ ]:
from dp_tester.generate_datasets import (
    generate_D_0_dataset,
    generate_adj_datasets,
    db_engine,
)
from dp_tester.constants import D_1
import pandas as pd

# generate the dataset and push it sql
generate_D_0_dataset()

# remove the user 0 to create the neighbor dataset and push it to sql under D_1 schema.
generate_adj_datasets(D_1, user_id=0)

# check the dataset pushed to sql.
engine = db_engine()
df_d0 = pd.read_sql("SELECT * FROM transactions", engine)
df_d1 = pd.read_sql(f'SELECT * FROM "{D_1}".transactions', engine)
print(len(df_d0), len(df_d1))

### Collecting DP Results

We use the `dp_results_from_sql_query` function to collect DP results. This function needs an object to execute sql queries to the database: `SqlAlchemyQueryExecutor`, an object to rewrite the query such the results will be differentially-private according to privacy parameters provided: `PyqrlewDpRewriter` does that and an object that changes tables name in the query in order to send it towards the $D_1$ dataset: `PyqrlewTableRenamer` is made for that. These objects follow protocols defined in `dp_tester/typing.py`.

The query is rewritten in the differentially private one and then is submitted multiple times to both $D_0$ and $D_1$ and the results collected and stored in a dictionary.

---

In [ ]:
from dp_tester.results_collector import dp_results_from_sql_query
from dp_tester.query_executors import SqlAlchemyQueryExecutor
from dp_tester.dp_rewriters import PyqrlewDpRewriter
from dp_tester.table_renamers import PyqrlewTableRenamer
from dp_tester.constants import D_0

query = "SELECT store_id, SUM(spent) AS spent FROM transactions GROUP BY store_id"
epsilon = 10.0
delta = 1e-4
runs = 10000
max_user_contributions_per_group = 10

query_executor = SqlAlchemyQueryExecutor()
dp_rewriter = PyqrlewDpRewriter(
    engine=query_executor.engine,
    max_privacy_unit_groups=max_user_contributions_per_group,
)
tables = ["users", "transactions"]
table_renamer = PyqrlewTableRenamer(dp_rewriter.dataset, tables)

results = dp_results_from_sql_query(
    non_dp_query=query,
    epsilon=epsilon,
    delta=delta,
    runs=runs,
    dp_rewriter=dp_rewriter,
    query_executor=query_executor,
    table_renamer=table_renamer,
    d_0=D_0,
    adjacent_ds=[D_1],
)

### Partitioning results

In our specific case, each result $r$ is a sequence of rows where each row is `(store_id, spent)` pair. Each `store_id` $g \in G = \{g_1, g_2, ..., g_M \}$ appears at most ones in the result since it is in the GROUP BY of the query. As in the example shown earlier, we can consider the result $r$ as a function that maps groups to their spent value:

$r: G \to \mathbb{R} \cup \{\emptyset\}$. We define the bucket function $p_g$ for a specific $g \in G$ as: $p_g(x) : \mathbb{R} \to \{1, 2, \dots, N, \emptyset\}$

where $p_g(x)$ determines the index of the bucket $I_j$ associated with the spent value $x$, or $\emptyset$ if $g$ is absent. Formally:

$$
p_g(r(g)) = \begin{cases}
j & \text{if } r(g) = x \text{ and } x \in I_j \text{ for some } j \in \{1, 2, \dots, N\}, \\
\emptyset & \text{otherwise }
\end{cases}
$$

Here, the of set of buckets $\{I_1, I_2, \dots, I_N\}$ is a set disjoint intervals. We can have one bucket functions like this per group. Such functions are then stored in a list called  `partition_vector` in the code. Notice that this approach is very flexible and it allows to test for any sort of manipulation of the results that can be mapped to an integer. For queries like the one we are using here, bucket functions are provided by `QuantityOverGroups.partition_vector()`.
The function `results_to_bucket_ids` takes as arguments the results and the list with bucket functions, iterates over the results and returns `t.List[t.Dict[str, t.List[int|None]]]`. Each element of the first level list is a dictionary associated with the outcome of a particular bucket function, thus the length of partition_vector is the same as the length of such list. Each key in the dictionary is dataset from which we colleted the results and the associated value is the list bucket indexes. We can now compute counts to estimate $\widehat{FP}_{\tau}, \widehat{FN}_{\tau}$ and compute $\hat{\epsilon}^{*}$.

In [ ]:
from dp_tester.generate_datasets import N_STORES
from dp_tester.partitioners import QuantityOverGroups
from dp_tester.analyzer import results_to_bucket_ids

NBINS = 20
partitioner = QuantityOverGroups(groups=list(range(N_STORES)))
partitioner.generate_buckets(results=results, n_bins=NBINS)
bucket_ids = results_to_bucket_ids(
    results=results, partition_vector=partitioner.partition_vector()
)

### Compute Empirical Epsilon

Once we have assigned each result to a specific bucket (under both $D_0$ and $D_1$), we obtain counts: $ c_j^{D_0}$ and $c_j^{D_1}$ for each bucket $j$. Let $C = \sum_{j} c_j^{D_0} = \sum_{j} c_j^{D_1}$ be the total number of experiments.

#### Sorting Buckets by Likelihood Ratio

To apply the Neyman–Pearson lemma, we consider the empirical likelihood ratio $\frac{c_j^{D_0}}{c_j^{D_1}}$. For simplicity, we sort buckets in descending order so those more indicative of $D_0$ (higher value) appear first, and those more indicative of $D_1$ (lower value) appear last:

$$ \frac{c_1^{D_0}}{c_1^{D_1}} \geq \frac{c_2^{D_0}}{c_2^{D_1}} \geq \dots \geq \frac{c_{N}^{D_0}}{c_{N}^{D_1}} $$

#### Choosing Thresholds $\tau_i$

Let $\tau_i$ represent a threshold set between bucket i and i+1. Intuitively:
- Buckets $ \leq i$  are more like $D_0$.
- Buckets  $> i$  are more like $D_1$.

According to the Neyman–Pearson principle, we reject $H_0$ if the data appear more indicative of $D_1$. Hence, the “reject H_0” region corresponds to buckets $j > i$.

#### Computing $FP$ and $FN$
- False Positive ($FP_{\tau_i}$): This occurs if we reject $H_0$ when $H_0$ is actually true. Under $H_0$, the probability of landing in bucket $j$ is estimated as $\frac{c_j^{D_0}}{C}$. Since rejecting $H_0$ involves buckets $j > i$:

$$ FP_{\tau_i} = \sum_{j > i} \frac{c_j^{D_0}}{C} = 1 - \sum_{j \leq i} \frac{c_j^{D_0}}{C}.$$

- False Negative ($FN_{\tau_i}$): This occurs if we fail to reject $H_0$ when $H_1$ is true. Under $H_1$, the probability of landing in bucket $j$ is $\frac{c_j^{D_1}}{C}$. Failing to reject $H_0$ corresponds to buckets $j \leq i$:

$$ FN_{\tau_i} = \sum_{j \leq i} \frac{c_j^{D_1}}{C}. $$ 


#### Computing $\hat{\epsilon}^{*}$
 
We can now apply the formula derived earlier:

$$ \hat{\epsilon}^{*} = \max_{\tau_i,\pi} \left[ \ln\left(\frac{1 - \delta - \widehat{FP_{\tau_i}}}{\widehat{FN_{\tau_i}}}\right), \ln\left(\frac{1 - \delta - \widehat{FN_{\tau_i}}}{\widehat{FP_{\tau_i}}}\right) \right] $$

We also include the symmetric case where $D_0$ and $D_1$ are swapped, ensuring both adding and removing a user are tested. This might add a second term in the $\\max$ operation, where $FN$ and $FP$ are switched.

#### Adjusting the Minimum Count Threshold

We impose a minimum count threshold $T$, requiring that $\sum_{j \leq i} c^{D_0}_j \geq T$ and $\sum_{j \leq i} c^{D_1}_j \geq T$ when computing the empirical $\epsilon$. This ensures that $FP$ and $FN$ are only computed for thresholds $\tau_i$ with sufficient observations to provide reliable estimates.


By following this procedure—partition results into buckets, sorting buckets according to the likelihood ratio, computing $\widehat{FP_{\tau_i}}$ and $\widehat{FN_{\tau_i}}$ and finally computing $\hat{\epsilon}^{*}$ you can empirically validate whether a mechanism meets its differential privacy guarantees.


---

In [ ]:
from dp_tester.analyzer import empirical_epsilon
from dp_tester.analyzer import counts_from_indexes

COUNT_THRESHOLD = 5

empirical_eps_per_group = {}
for i, partition_bucket_ids in enumerate(bucket_ids):
    counts_d_0 = counts_from_indexes(
        indexes=partition_bucket_ids[D_0], max_bucket_length=len(partitioner.buckets)
    )
    counts_d_1 = counts_from_indexes(
        indexes=partition_bucket_ids[D_1], max_bucket_length=len(partitioner.buckets)
    )
    empirical_eps_per_group[i] = empirical_epsilon(
        counts_d_0,
        counts_d_1,
        delta=delta,
        counts_threshold=COUNT_THRESHOLD,
        plot=False,
    )
all_eps = list(empirical_eps_per_group.values())
max_eps = max(all_eps)

print(f"Epsilon used during the experiment: {epsilon}")
print(f"Max empirical epsilon found: {max_eps}")
print(f"Did the test passed? {max_eps <= epsilon}")

## Conclusions

In this notebook, we demonstrated a practical method for testing differential privacy mechanisms using hypothesis testing. By leveraging the dp_testing library, we showed how to generate realistic datasets, apply differential privacy, and empirically evaluate the resulting privacy guarantees. This approach not only helped us detect and correct a flaw in one of our DP implementations, but it also provides a general framework that developers and researchers can use to ensure their differential privacy solutions are both robust and reliable.


## References

- [Differencial Privacy](https://maxkasy.github.io/home/files/other/ML_Econ_Oxford/differential_privacy.pdf)
- [Wilson et al.](https://arxiv.org/abs/1909.01917)
- [Kairouz et al.](https://arxiv.org/abs/1311.0776)
- [Milad Nasr et al.](https://arxiv.org/abs/2101.04535)